In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import tikzplotlib
import os

# SPECTROGRAM ANN TYPES

In [2]:
SPECTROGRAMS = ['spectrograms', 'mel_spectrograms', 'mfcc_spectrograms', 'cqt_spectrograms']
VARIABLES = ['n_fft', 'hop_length', 'win_length', 'n_mels', 'n_mfcc']

In [3]:
FILEPATH = "../IST non-JNB results/spectrogram_tensors"

In [4]:
columns = ['type', 'n_fft', 'hop_len', 'win_len', 'n_mel', 'n_mfcc', 'Accuracy', 'Epoch', 'Test Loss', 'F1', 'Recall', 'Precision', 'AUC-ROC', 'Confusion Matrix', 'Train Accuracy', 'Train Loss', 'Train F1', 'Train Recall', 'Train Precision', 'Train AUC-ROC', 'Train Confusion Matrix']
spectrograms = pd.DataFrame(columns=columns)

for folder in os.listdir(f"{FILEPATH}/mel_spectrograms"):
    if folder != ".DS_Store":
        # Load data
        test = pd.read_csv(f"{FILEPATH}/mel_spectrograms/{folder}/test.csv", index_col=None)
        train = pd.read_csv(f"{FILEPATH}/mel_spectrograms/{folder}/train.csv", index_col=None)

        test.drop(test.columns[0], axis=1, inplace=True)
        train.drop(train.columns[0], axis=1, inplace=True)

        # Find index of highest accuracy in test set
        idx_test = test['Accuracy'].idxmax()

        # Get row with highest accuracy in test set
        highest_test_row = test.loc[idx_test].copy()

        # Rename and drop columns in the test row
        highest_train_row = test.loc[idx_test].drop(columns=["Epoch"]).rename({"Accuracy": "Train Accuracy", "Test Loss": "Train Loss", "F1": "Train F1", "Recall": "Train Recall", "Precision": "Train Precision", "AUC-ROC": "Train AUC-ROC", "Confusion Matrix": "Train Confusion Matrix"}).copy()

        # Split folder name to extract information
        info = folder.split('-')
        data_info = {'type': 'mel', 'n_fft': info[0], 'hop_len': info[1], 'win_len': info[2], 'n_mel': info[3], 'n_mfcc': 0}

        # Assign values to additional columns
        for col, value in data_info.items():
            highest_test_row[col] = value
            highest_train_row[col] = value

        # Concatenate highest_test_row and highest_train_row along the column axis
        full_row = pd.concat([highest_test_row, highest_train_row], axis=1)


        # Append full_row to spectrograms DataFrame
        spectrograms = pd.concat([spectrograms, full_row])


for folder in os.listdir(f"{FILEPATH}/mfcc_spectrograms"):
    if folder != ".DS_Store":
        # Load data
        test = pd.read_csv(f"{FILEPATH}/mfcc_spectrograms/{folder}/test.csv", index_col=None)
        train = pd.read_csv(f"{FILEPATH}/mfcc_spectrograms/{folder}/train.csv", index_col=None)

        # Find index of highest accuracy in test set
        idx_test = test['Accuracy'].idxmax()

        # Get row with highest accuracy in test set
        highest_test_row = test.loc[idx_test].copy()

        # Rename and drop columns in the test row
        highest_train_row = test.loc[idx_test].drop(columns=["Epoch"]).rename({"Accuracy": "Train Accuracy", "Test Loss": "Train Loss", "F1": "Train F1", "Recall": "Train Recall", "Precision": "Train Precision", "AUC-ROC": "Train AUC-ROC", "Confusion Matrix": "Train Confusion Matrix"}).copy()

        # Split folder name to extract information
        info = folder.split('-')
        data_info = {'type': 'mfcc', 'n_fft': info[0], 'hop_len': info[1], 'win_len': info[2], 'n_mel': info[3], 'n_mfcc': info[4]}

        # Assign values to additional columns
        for col, value in data_info.items():
            highest_test_row[col] = value
            highest_train_row[col] = value

        # Concatenate highest_test_row and highest_train_row along the column axis
        full_row = pd.concat([highest_test_row, highest_train_row], axis=1)

        # Append full_row to spectrograms DataFrame
        spectrograms = pd.concat([spectrograms, full_row])


for folder in os.listdir(f"{FILEPATH}/cqt_spectrograms"):
    if folder != ".DS_Store":
        # Load data
        test = pd.read_csv(f"{FILEPATH}/cqt_spectrograms/{folder}/test.csv", index_col=None)
        train = pd.read_csv(f"{FILEPATH}/cqt_spectrograms/{folder}/train.csv", index_col=None)

        # Find index of highest accuracy in test set
        idx_test = test['Accuracy'].idxmax()

        # Get row with highest accuracy in test set
        highest_test_row = test.loc[idx_test].copy()

        # Rename and drop columns in the test row
        highest_train_row = test.loc[idx_test].drop(columns=["Epoch"]).rename({"Accuracy": "Train Accuracy", "Test Loss": "Train Loss", "F1": "Train F1", "Recall": "Train Recall", "Precision": "Train Precision", "AUC-ROC": "Train AUC-ROC", "Confusion Matrix": "Train Confusion Matrix"}).copy()

        # Split folder name to extract information
        info = folder.split('-')
        data_info = {'type': 'cqt', 'n_fft': 0, 'hop_len': info[0], 'win_len': 0, 'n_mel': 0, 'n_mfcc': 0}

        # Assign values to additional columns
        for col, value in data_info.items():
            highest_test_row[col] = value
            highest_train_row[col] = value

        full_row = pd.concat([highest_test_row, highest_train_row], axis=1)

        # Append full_row to spectrograms DataFrame
        spectrograms = pd.concat([spectrograms, full_row])

print(spectrograms)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

# INPUT ENCODING -- THRESHOLD BASED AND STEP-FORWARD

In [11]:
THRESHOLDS = [0.01, 0.025, 0.05, 0.10, 0.20, 0.50]
OFF_SPIKES = [True, False]
SPECTROGRAMS = ['mel_spectrograms', 'mfcc_spectrograms']

columns = ['Spectrogram', 'Threshold', 'Off Spike', 'Accuracy', 'Epoch', 'Test Loss', 'Max F1', 'Max AUC-ROC', 'Max Confusion Matrix', 'Train Accuracy', 'Train Loss', 'Train F1', 'Train AUC-ROC', 'Train Confusion Matrix']

FILEPATH = "../IST non-JNB results/input_encoding/step_forward"

max_thresholds = pd.DataFrame(columns=columns)

for spectrogram_type in SPECTROGRAMS:
    for threshold in THRESHOLDS:
        for off_spike in OFF_SPIKES:
            test = pd.read_csv(f"{FILEPATH}/{threshold}_{off_spike}/{spectrogram_type}/test.csv", index_col=None)
            train = pd.read_csv(f"{FILEPATH}/{threshold}_{off_spike}/{spectrogram_type}/train.csv", index_col=None)

            max_f1_index = test['F1'].idxmax()
            max_train = train.loc[max_f1_index]
            max_test = test.loc[max_f1_index]
            data_info = {'Spectrogram': spectrogram_type, 'Threshold': threshold, 'Off Spike': off_spike, 'Accuracy': max_test['Accuracy'], 'Epoch': max_test['Epoch'], 'Test Loss': max_test['Test Loss'], 
                         'Max F1': max_test['F1'], 'Max AUC-ROC': max_test['AUC-ROC'], 'Max Confusion Matrix': max_test['Confusion Matrix'], 'Train Accuracy': max_train['Accuracy'], 'Train Loss': max_train['Test Loss'], 'Train F1': max_train['F1'], 'Train AUC-ROC': max_train['AUC-ROC'], 'Train Confusion Matrix': max_train['Confusion Matrix']}
            max_thresholds = max_thresholds._append(data_info, ignore_index=True)
        

max_thresholds.to_csv('max_step_forward.csv')


FILEPATH = "../IST non-JNB results/input_encoding/time_contrast"

max_thresholds = pd.DataFrame(columns=columns)

for spectrogram_type in SPECTROGRAMS:
    for threshold in THRESHOLDS:
        for off_spike in OFF_SPIKES:
            test = pd.read_csv(f"{FILEPATH}/{threshold}_{off_spike}/{spectrogram_type}/test.csv", index_col=None)
            train = pd.read_csv(f"{FILEPATH}/{threshold}_{off_spike}/{spectrogram_type}/train.csv", index_col=None)

            max_f1_index = test['F1'].idxmax()
            max_train = train.loc[max_f1_index]
            max_test = test.loc[max_f1_index]
            data_info = {'Spectrogram': spectrogram_type, 'Threshold': threshold, 'Off Spike': off_spike, 'Accuracy': max_test['Accuracy'], 'Epoch': max_test['Epoch'], 'Test Loss': max_test['Test Loss'], 
                         'Max F1': max_test['F1'], 'Max AUC-ROC': max_test['AUC-ROC'], 'Max Confusion Matrix': max_test['Confusion Matrix'], 'Train Accuracy': max_train['Accuracy'], 'Train Loss': max_train['Test Loss'], 'Train F1': max_train['F1'], 'Train AUC-ROC': max_train['AUC-ROC'], 'Train Confusion Matrix': max_train['Confusion Matrix']}
            max_thresholds = max_thresholds._append(data_info, ignore_index=True)
        

max_thresholds.to_csv('max_thresholds.csv')


/var/folders/mq/y1mzq23j39104lm0kgrz14180000gn/T/ipykernel_82258/1410776555.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  max_thresholds = max_thresholds._append(data_info, ignore_index=True)
/var/folders/mq/y1mzq23j39104lm0kgrz14180000gn/T/ipykernel_82258/1410776555.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  max_thresholds = max_thresholds._append(data_info, ignore_index=True)


# INPUT ENCODING -- RATE (COUNT NUMBER)

# INPUT ENCODING -- AUTOENCODER

# ARCHITECTURE -- MLP-SNN (DECAY RATE)